In [1]:
#Import commands

from flask import Flask, flash, redirect, render_template, request, session, abort
import pandas as pd
import numpy as np
import json
import xlrd
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#Main code

#1. Define app
app= Flask(__name__)


#2. Create route, homepage and pull in data (Please change path to data as you see fit)
@app.route("/",methods=["GET","POST"])
def homepage():
#kbn 2018/12/12-Start- Call value from HTML form
  data1 = request.form.get('question1_field','Education')
  params = data1
#kbn 2018/12/12-End
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  datasource1 = pd.read_excel("C:/Users/Public/Rfiles/Data_for_Similarity_New.xlsx")
  datasource1 = pd.DataFrame(datasource1)

#3. Divide the input words into multiple words and write a loop to filter dataset for words in the "Text" column
#kbn 2018/12/12-Start- Here divide inputs into multiple words and filter data using words to narrow sample size
  words = data1.split()

  datasource2 = []
  for i in words:
      print(i)
      d2 = datasource1[datasource1['Text'].str.contains(str(i))]
      datasource2.append(d2)

  datasource2 = pd.concat(datasource2, ignore_index=True)

  datasource2 = pd.DataFrame(datasource2)
  datasource2=datasource2.drop_duplicates()
  datasource2=datasource2.reset_index()
  #print(datasource.head())
#kbn 2018/12/12-End

#4. Start computation of similarilty score here
  datasource=datasource2
  index_source = 0
  max_simi = 0

  for i in range(0, len(datasource)):
      documents = []
      documents.append(params)
      documents.append(datasource.loc[i, 'Text'])
# kbn 2018/12/12-Start- add stop_words and ngram_range to the TfidVectorizer below
      tfidf = TfidfVectorizer(stop_words="english", ngram_range=(1, 4)).fit_transform(documents)
# kbn 2018/12/12-End
      # no need to normalize, since Vectorizer will return normalized tf-idf
      pairwise_similarity = tfidf * tfidf.T
      # Add stop word, Add ngram

      temp = pairwise_similarity.A[0][1]

      simiscore = "{0:.4f}".format(temp)
      print(simiscore)
      datasource.loc[i, 'Simiscore'] = simiscore

  datasource['Rank'] = datasource['Simiscore'].rank(method='average', ascending=False)
  datasource = datasource.sort_values('Rank', ascending=True)
  datasource = datasource.head(10)
  datasource = datasource.reset_index()

  # select some needed columns to display
  datadisplay = datasource[
      ["Rank", "ProjectName", "Organization", "Country", "Sector", "Year", "Practitioner", "ProjectID","Text",
       "Simiscore"]]
  datadisplay.Rank = datadisplay.Rank.astype(np.int64)

  datadisplay=datadisplay.to_json(orient="records")

#5. Define return here (I am returning as json since that is the best for the html file)
  return render_template('documents.html', docs=json.loads(datadisplay))

#6. Run app below
#debugger turned off below since this is a jupyter notebook
if __name__ == "__main__":
  app.debug = False
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
